In [3]:
import sys
import pandas as pd
import psycopg2

In [4]:
# connect postgreSQL
sys.path.append('..')
import connection_info
users = connection_info.users
dbnames = connection_info.dbnames
passwords = connection_info.passwords
conn = psycopg2.connect(" user=" + users +" dbname=" + dbnames +" password=" + passwords)

## 4-1 マスタテーブルの結合

### 4-1 Not Awesome

In [5]:
reserve4_1NA = pd.read_sql(
sql=
    """
-- 予約テーブルとホテルテーブルをすべて結合
WITH rsv_and_hotel_tb AS(
	SELECT
		-- 必要な列の抽出
	  rsv.reserve_id, rsv.hotel_id, rsv.customer_id,
	  rsv.reserve_datetime, rsv.checkin_date, rsv.checkin_time,
		rsv.checkout_date, rsv.people_num, rsv.total_price,
	  hotel.base_price, hotel.big_area_name, hotel.small_area_name,
	  hotel.hotel_latitude, hotel.hotel_longitude, hotel.is_business

	-- 結合元となるreserve_tbを選択、テーブルの短縮名をrsvに設定
	FROM work.reserve_tb rsv

	-- 結合するhotel_tbを選択、テーブルの短縮名をhotelに設定
	INNER JOIN work.hotel_tb hotel
		-- 結合の条件を指定、hotel_idが同じレコード同士を結合
	  ON rsv.hotel_id = hotel.hotel_id
)
-- 結合したテーブルから条件に適合するデータのみ抽出
SELECT * FROM rsv_and_hotel_tb

-- is_businessのデータのみ抽出
WHERE is_business is True

	-- people_numが1人のデータのみ抽出
  AND people_num = 1
""",
con=conn)

In [6]:
reserve4_1NA.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
2,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.329097,140.698165,True
3,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.231842,140.797268,True
4,r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.233985,140.795603,True


### 4-1 Awesome

In [7]:
reserve4_1Awe = pd.read_sql(
sql=
"""
SELECT
	-- 必要な列の抽出
	rsv.reserve_id, rsv.hotel_id, rsv.customer_id,
  rsv.reserve_datetime, rsv.checkin_date, rsv.checkin_time, rsv.checkout_date,
  rsv.people_num, rsv.total_price,
  hotel.base_price, hotel.big_area_name, hotel.small_area_name,
  hotel.hotel_latitude, hotel.hotel_longitude, hotel.is_business

FROM work.reserve_tb rsv

-- INNER JOIN と JOIN は同じ
JOIN work.hotel_tb hotel
  ON rsv.hotel_id = hotel.hotel_id

-- ホテルテーブルからビジネスホテルのデータのみ抽出
WHERE hotel.is_business is True

	-- 予約テーブルからビジネスホテルのデータのみ抽出
  AND rsv.people_num = 1
""",
con=conn)

In [8]:
reserve4_1Awe.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,base_price,big_area_name,small_area_name,hotel_latitude,hotel_longitude,is_business
0,r7,h_256,c_1,2017-12-29 10:38:36,2018-01-25,10:30:00,2018-01-28,1,103500,34500,C,C-1,38.237294,140.696131,True
1,r11,h_183,c_2,2016-11-19 12:49:10,2016-12-08,11:00:00,2016-12-11,1,29700,9900,G,G-4,33.595248,130.633567,True
2,r13,h_223,c_2,2017-10-19 03:03:30,2017-10-21,09:30:00,2017-10-23,1,137000,68500,C,C-2,38.329097,140.698165,True
3,r18,h_132,c_3,2016-10-22 02:18:48,2016-11-12,12:00:00,2016-11-13,1,20400,20400,C,C-1,38.231842,140.797268,True
4,r25,h_277,c_4,2016-03-28 07:17:34,2016-04-07,10:30:00,2016-04-10,1,39300,13100,C,C-1,38.233985,140.795603,True


In [9]:
len(reserve4_1Awe)

446

## 4-2 条件に応じた結合テーブルの切り替え

### 4-2 Awesome

In [10]:
reserve4_2 = pd.read_sql(
sql=
    """
-- 予約テーブルとホテルテーブルをすべて結合
-- small_area_nameごとにホテル数をカウント、結合キーを判定するためのテーブル
WITH small_area_mst AS(
  SELECT
    small_area_name,

    -- 20件以上であればjoin_area_idをsmall_area_nameとして設定
    -- 20件未満であればjoin_area_idをbig_area_nameとして設定
    -- -1は、自ホテルを引いている
    CASE WHEN COUNT(hotel_id)-1 >= 20
			THEN small_area_name ELSE big_area_name END AS join_area_id

  FROM work.hotel_tb
  GROUP BY big_area_name, small_area_name
)
-- recommend_hotel_mstはレコメンド候補のためのテーブル
, recommend_hotel_mst AS(
  -- join_area_idをbig_area_nameとしたレコメンド候補マスタ
  SELECT
    big_area_name AS join_area_id,
    hotel_id AS rec_hotel_id
  FROM work.hotel_tb

  -- unionで、テーブル同士を連結
  UNION

  -- join_area_idをsmall_area_nameとしたレコメンド候補マスタ
  SELECT
    small_area_name AS join_area_id,
    hotel_id AS rec_hotel_id
  FROM work.hotel_tb
)
SELECT
  hotels.hotel_id,
  r_hotel_mst.rec_hotel_id

-- レコメンド元のhotel_tbを読み込み
FROM work.hotel_tb hotels

-- 各ホテルのレコメンド候補の対象エリアを判断するためにsmall_area_mstを結合
INNER JOIN small_area_mst s_area_mst
  ON hotels.small_area_name = s_area_mst.small_area_name

-- 対象エリアのレコメンド候補を結合する
INNER JOIN recommend_hotel_mst r_hotel_mst
  ON s_area_mst.join_area_id = r_hotel_mst.join_area_id

  -- レコメンド候補から自分ホテルを除く
  AND hotels.hotel_id != r_hotel_mst.rec_hotel_id
""",
con=conn)

In [11]:
reserve4_2.head()

,hotel_id,rec_hotel_id
0,h_1,h_275
1,h_1,h_77
2,h_1,h_79
3,h_1,h_91
4,h_1,h_40


## 4-3 過去データの結合

### 4-3a Awesome

In [12]:
reserve4_3a = pd.read_sql(
sql=
    """
SELECT
	*,

	-- LAG関数を利用し、2件前のtotal_priceをbefore_priceとして取得
	-- LAG関数によって参照する際のグループをcustomer_idに指定
	-- LAG関数によって参照する際のグループ内のデータをreserve_datetimeの古い順に指定
  LAG(total_price, 2) OVER
	(PARTITION BY customer_id ORDER BY reserve_datetime) AS before_price

FROM work.reserve_tb
""",
con=conn)

In [13]:
reserve4_3a.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,before_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,97200.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,20600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,33600.0


### 4-3b 過去n件の合計値
### Awesome

In [14]:
reserve4_3b = pd.read_sql(
sql=
    """
SELECT
  *,
  CASE WHEN

    -- COUNT関数で何件の合計を計算したかをカウントし、3件あるのかを判定
		-- BETWEEN句で2件前から
  	COUNT(total_price) OVER
		(PARTITION BY customer_id ORDER BY reserve_datetime ROWS
		 BETWEEN 2 PRECEDING AND CURRENT ROW) = 3

  THEN

    -- 自身を含めた3件の合計金額を計算
  	SUM(total_price) OVER
		(PARTITION BY customer_id ORDER BY reserve_datetime ROWS
		 BETWEEN 2  PRECEDING AND CURRENT ROW)

  ELSE NULL END AS price_sum

FROM work.reserve_tb
""",
con=conn)

In [15]:
reserve4_3b.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_sum
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,NaN
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,NaN
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,151400.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,248600.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,296100.0
